In [ ]:
import sys
import numpy as np
import copy
import skfuzzy as fuzz
import skfuzzy.control as ctrl
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
import wandb

In [ ]:
!wandb init

In [ ]:
CA = False
total_episode = 3000
TTC_threshold = 4.0002
base_name = f'origin_{TTC_threshold}_NOCA'

In [ ]:
def Fuzzy_reward(data):#6 相对安全 collisions: 0 Average TTC: 35.15633820442351 Average Headway: 0.9781254979624681 Average Jerk: 0.27031795829138777 Average Distance: 11.181573238108232 Average Rel_Speed: 1.250587707943024
 #collisions: 0 Average TTC: 247.9903994822236 Average Headway: 1.2830073657006325 Average Jerk: 0.8863629586605888 Average Distance: 33.30417367109366 Average Rel_Speed: 2.5152311733037798



      train=data
      F_Data_speed=[]
      F_Data_distance=[]
      F_Data_acceleration=[]
      L_Data_speed=[]
      for i in range(len(train)):
        F_Data_speed.append(np.mean(train[i,0][:,1]))
        F_Data_distance.append(np.mean(train[i,0][:,0]))
        F_Data_acceleration.append(np.mean((np.diff(train[i,0][:,1])/0.1)))
        L_Data_speed.append(np.mean((train[i,0][:,1]-train[i,0][:,3])))
      # print("????",F_Data_acceleration)

      min_max_scaler_speed = MinMaxScaler(feature_range=(0,10))
      min_max_scaler_speed.fit([[0], [30]])
      min_max_scaler_speed_L = MinMaxScaler(feature_range=(0,10))
      min_max_scaler_speed_L.fit([[-30], [30]])
      min_max_scaler_distance = MinMaxScaler(feature_range=(0,10))
      min_max_scaler_distance.fit([[0], [30]])
      min_max_scaler_acceleration = MinMaxScaler(feature_range=(0,10))
      min_max_scaler_acceleration.fit([[-3], [3]])
      # print("????www")


      F_Data_speed = min_max_scaler_speed.transform(np.array(F_Data_speed).reshape(-1, 1))
      F_Data_speed = np.squeeze(F_Data_speed,axis=1)
      L_Data_speed = min_max_scaler_speed_L.transform(np.array(L_Data_speed).reshape(-1, 1))
      L_Data_speed = np.squeeze(L_Data_speed,axis=1)
      F_Data_distance = min_max_scaler_distance.transform(np.array(F_Data_distance).reshape(-1,1))
      F_Data_distance = np.squeeze(F_Data_distance,axis=1)
      F_Data_acceleration = min_max_scaler_acceleration.transform(np.array(F_Data_acceleration).reshape(-1,1))
      F_Data_acceleration = np.squeeze(F_Data_acceleration,axis=1)

      # print((F_Data_speed),F_Data_distance)


      Speed_range_L=np.arange(0,10,0.1)
      Speed_range=np.arange(0,10,0.1)
      Distance_range=np.arange(0,10,0.1)
      Acceleration_range=np.arange(0,10,0.1)
      Reward_range=np.arange(0,10)

      speed_mu=np.mean(F_Data_speed)
      speed_sigma=np.std(F_Data_speed)

      L_speed_mu=np.mean(L_Data_speed)
      L_speed_sigma=np.std(L_Data_speed)

      distance_mu=np.mean(F_Data_distance)
      distance_sigma=np.std(F_Data_distance)

      acceleration_mu=np.mean(F_Data_acceleration)
      acceleration_sigma=np.std(F_Data_acceleration)


      x_speed_L=ctrl.Antecedent(Speed_range_L, 'speed_L')
      x_speed=ctrl.Antecedent(Speed_range, 'speed')
      x_gap=ctrl.Antecedent(Distance_range, 'distance')
      x_acceleration=ctrl.Antecedent(Acceleration_range, 'acceleration')
      y_reward=ctrl.Consequent(Reward_range, 'reward')


      # x_speed_L['N']=fuzz.gaussmf(x_speed_L.universe,L_speed_mu,L_speed_sigma)
      # x_speed_L['N']=fuzz.trimf(Speed_range,L_speed_mu,L_speed_sigma)

      x_speed_L['N']=fuzz.trimf(x_speed_L.universe,[3.9,5,6.0])

      x_speed_L['S']=fuzz.trimf(x_speed_L.universe,[0,0,4.1])

      x_speed_L['B']=fuzz.trimf(x_speed_L.universe,[5.8,10,10])


      x_speed['N']=fuzz.trimf(Speed_range,[0.7,1.3,2])

      x_speed['S']=fuzz.trimf(Speed_range,[0,0,0.8])

      x_speed['B']=fuzz.trimf(Speed_range,[1.9,10,10])


      # x_gap['N']=fuzz.gaussmf(x_gap.universe,distance_mu,distance_sigma)

      x_gap['N']=fuzz.trimf(x_gap.universe,[3,5,7])

      x_gap['S']=fuzz.trimf(x_gap.universe,[0,0,3.5])

      x_gap['B']=fuzz.trimf(x_gap.universe,[6.5,10,10])


      # x_acceleration['N']=fuzz.gaussmf(x_acceleration.universe,acceleration_mu,acceleration_sigma)

      x_acceleration['N']=fuzz.trimf(x_acceleration.universe,[5.1,6,7.4])

      x_acceleration['S']=fuzz.trimf(x_acceleration.universe,[0,0,5.4])

      x_acceleration['B']=fuzz.trimf(x_acceleration.universe,[7.1,10,10])


      y_reward['N']=fuzz.trimf(y_reward.universe,[2,5,8])

      y_reward['S']=fuzz.trimf(y_reward.universe,[0,0,3])

      y_reward['B']=fuzz.trimf(y_reward.universe,[7,10,10])





      # 设定输出powder的解模糊方法——质心解模糊方式

      y_reward.defuzzify_method='centroid'

      rule0 = ctrl.Rule(antecedent=(


                                  (x_speed_L['S'] & x_acceleration['S'] ) |

                                  (x_speed_L['S'] & x_acceleration['B'] & x_gap['N']) |
                                  (x_speed_L['N'] & x_acceleration['S'] & x_gap['N']) |
                                  (x_speed_L['N'] & x_acceleration['B'] & x_gap['N']) |
                                  (x_speed_L['S'] & x_acceleration['B'] & x_gap['B']) ),

                      consequent=y_reward['N'], label='rule N')

    # 输出为M的规则

      rule1 = ctrl.Rule(antecedent=(
                                    (x_speed_L['S'] & x_acceleration['B'] & x_gap['S']) |
                                    (x_speed_L['N'] & x_gap['S']) |
                                    (x_speed_L['N'] & x_gap['B']) |
                                    (x_speed_L['B'] & x_acceleration['B']) |
                                    (x_speed_L['B'] & x_acceleration['N'])),

                        consequent=y_reward['S'], label='rule S')

      # 输出为P的规则

      rule2 = ctrl.Rule(antecedent=(



                                    (x_speed_L['N'] & x_acceleration['N'] & x_gap['N']) |
                                    (x_speed_L['S'] & x_acceleration['N']) |
                                    (x_speed_L['B'] & x_acceleration['S']  )),

                        consequent=y_reward['B'], label='rule B')

      system = ctrl.ControlSystem(rules=[rule0, rule1, rule2])
      sim = ctrl.ControlSystemSimulation(system)

      return sim,min_max_scaler_speed,min_max_scaler_distance,min_max_scaler_acceleration,min_max_scaler_speed_L

In [ ]:
import time
def Model_test(model,testNum,test,sim,speed,dis,acc,rel):
  ddpg=model
  total_episode = testNum
  rolling_window = 100
  max_rolling_score = np.float('-inf')
  max_score = np.float('-inf')
  collision_train = 0
  episode_score = np.zeros(total_episode)  # average score of each car following event
  rolling_score = np.zeros(total_episode)
  cum_collision_num = np.zeros(total_episode)
  ttc_score = np.zeros(total_episode)
  headway_score = np.zeros(total_episode)
  jerk_score =np.zeros(total_episode)
  space_gap=np.zeros(total_episode)

  score_safe = np.zeros(total_episode)
  score_efficiency = np.zeros(total_episode)
  score_comfort = np.zeros(total_episode)
  F_V = np.zeros(total_episode)
  Gap = np.zeros(total_episode)

  sim_data = []
  reward_data = []

  start_time = time.time()
  for i in tqdm(range(total_episode)):
      data = test[i, 0]
      s = env.reset(data)
      score = 0
      score_s, score_e, score_c ,score_TTC, score_Headway, score_jerk,space= 0, 0, 0, 0, 0, 0, 0   # part objective scores
      F_s,g_space=0,0
      while True:
          # Add exploration noise
          a = ddpg.choose_action(s)[0]
          F_s+=env.s[1]
          # print(env.s[1]+env.s[2])
          g_space+=env.s[0]
          # if a>3:
          #   a=3
          # elif a<-3:
          #   a=-3
          # else:
          #   pass
          if False:
                state = env.s
                space, svSpd, relSpd = state
                lvSpd = svSpd + relSpd
                RT = 1  # reaction time
                a_C_2 = 2
                a_C_1 = 1
                a_cc=((svSpd**2-lvSpd**2))/(2*space)
                if abs(a)>abs(a_cc):
                  if a>0:
                    a=abs(a_cc)

                  else:
                    a=-abs(a_cc)
                # CD_2 = svSpd * RT + (svSpd**2-lvSpd**2)/(2*a_C_2)
                # CD_1 = svSpd * RT + (svSpd**2-lvSpd**2)/(2*a_C_1)
                # # if space > CD_2 and abs(a)>a_C_2 :
                # #   print(11)
                # #   if a>0:
                # #     a=2
                # #   else:
                # #     a=-2

                # if space > CD_1 and abs(a)>a_C_1 :
                #   if a>0:
                #     a=1
                #   else:
                #     a=-1
          if CA:
              # add collision avoidance guidance
              state = env.s
              space, svSpd, relSpd = state
              lvSpd = svSpd + relSpd
              RT = 1  # reaction time
              SD = svSpd * RT + (svSpd ** 2 - lvSpd ** 2) / (2 * a_bound)

              if space < SD:
                  a = -a_bound

          s_, r, done, _,r_info = env.step(a,sim,speed,dis,acc,rel)

          reward_data.append(r_info)

          s = s_
          score += r
          score_TTC+= r_info[0]
          score_Headway+= r_info[1]
          score_jerk+= np.abs(r_info[2])
          score_s += r_info[3]
          score_e += r_info[4]
          score_c += r_info[5]
          space += r_info[6]

          if done:
              duration = data.shape[0]
              score /= duration  # normalize with respect to car-following length
              score_s /= duration
              score_e /= duration
              score_c /= duration
              score_TTC/= duration
              score_Headway/= duration
              score_jerk/= duration
              space/= duration
              F_s /= duration
              g_space /= duration
              if env.isCollision == 1:
                  collision_train += 1
              break

      # store traffic simulation data
      sim_data.append(np.vstack((env.LVSpdData, env.SimSpeedData, env.SimSpaceData)).T)


      # record episode results
      episode_score[i] = score
      score_safe[i] = score_s
      score_efficiency[i] = score_e
      score_comfort[i] = score_c
      rolling_score[i] = np.mean(episode_score[max(0, i - rolling_window + 1):i + 1])
      cum_collision_num[i] = collision_train
      ttc_score[i] =score_TTC
      headway_score[i] = score_Headway
      jerk_score[i] =score_jerk
      space_gap[i]=space
      F_V[i]=F_s
      Gap[i]=g_space
      if max_score < score:
          max_score = score

      if rolling_score[i] > max_rolling_score:
          max_rolling_score = rolling_score[i]

      sys.stdout.write(
          f'''\r Test with test data, Episode {i}, Score: {score:.2f}, Rolling score: {rolling_score[i]:.2f}, Max score: {max_score:.2f}, Max rolling score: {max_rolling_score:.2f}, collisions: {collision_train}   ''')
      sys.stdout.flush()
  end_time = time.time()
  print('Running time:', end_time - start_time)
  print('Running time per event:', (end_time - start_time)/testNum)
  #save simulation data to npy file
  np.save(f'{base_name}_traj_test.npy', sim_data)
  reward_data = np.array(reward_data)
  np.save(f'{base_name}_rinfo_test.npy', reward_data)
  np.savez(f'{base_name}_score_test.npz', episode_score, score_safe, score_efficiency, score_comfort, rolling_score)
  # Result Measure
  print("-"*50,"MeasureResult","-"*50)
  t=np.delete(ttc_score, np.where(ttc_score <=0  ))
  h=np.delete(headway_score, np.where(headway_score >=100 ))
  # k=np.delete(t, np.where(t >10000   ))

  dis=[]
  rel_speed=[]
  for i in range(len(test)):
    dis.append(np.mean(sim_data[i][:,2]))
    rel_speed.append(np.mean(np.abs(sim_data[i][:,1]-sim_data[i][:,0])))
  print('collisions:',collision_train,"Average TTC:",np.mean(t),"Average Headway:",np.mean(h),"Average Jerk:",np.mean(jerk_score),"Average Distance:",np.mean(dis),"Average Rel_Speed:",np.mean(rel_speed))
  print()
  print("-"*50,"END","-"*50)
  print()

In [ ]:
import tensorflow as tf
import random
import numpy as np
from simulation_env import Env
import scipy.io as sio
import pickle as pk
import sys
import os
import wandb
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from tqdm import tqdm_notebook as tqdm
import warnings

warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

get_ipython().run_line_magic('matplotlib', 'inline')

#####################  hyper parameters  ####################
LR_A = 0.001  # learning rate for actor
LR_C = 0.001  # learning rate for critic
GAMMA = 0.9  # reward discount
REPLACE_ITER_A = 500
REPLACE_ITER_C = 300
MEMORY_CAPACITY = 20000  # 7000
BATCH_SIZE = 1024

###############################  Actor  ####################################
class DDPG(object):
    def __init__(self, a_dim, s_dim, a_bound, ):
        self.memory = np.zeros((MEMORY_CAPACITY, s_dim * 2 + a_dim + 1), dtype=np.float32)
        self.pointer = 0
        self.sess = tf.compat.v1.Session()
        self.a_replace_counter, self.c_replace_counter = 0, 0

        self.a_dim, self.s_dim, self.a_bound = a_dim, s_dim, a_bound,

        tf.compat.v1.disable_eager_execution()
        self.S = tf.compat.v1.placeholder(tf.float32, [None, s_dim], 's')
        self.S_ = tf.compat.v1.placeholder(tf.float32, [None, s_dim], 's_')
        self.R = tf.compat.v1.placeholder(tf.float32, [None, 1], 'r')

        with tf.compat.v1.variable_scope('Actor'):
            self.a = self._build_a(self.S, scope='eval', trainable=True)
            a_ = self._build_a(self.S_, scope='target', trainable=False)
        with tf.compat.v1.variable_scope('Critic'):
            q = self._build_c(self.S, self.a, scope='eval', trainable=True)
            q_ = self._build_c(self.S_, a_, scope='target', trainable=False)

        # networks parameters
        self.ae_params = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.GLOBAL_VARIABLES, scope='Actor/eval')
        self.at_params = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.GLOBAL_VARIABLES, scope='Actor/target')
        self.ce_params = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.GLOBAL_VARIABLES, scope='Critic/eval')
        self.ct_params = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.GLOBAL_VARIABLES, scope='Critic/target')

        q_target = self.R + GAMMA * q_
        # in the feed_dic for the td_error, the self.a should change to actions in memory
        td_error = tf.compat.v1.losses.mean_squared_error(labels=q_target, predictions=q)
        self.ctrain = tf.compat.v1.train.AdamOptimizer(LR_C).minimize(td_error, var_list=self.ce_params)

        a_loss = - tf.compat.v1.reduce_mean(q)  # maximize the q
        self.atrain = tf.compat.v1.train.AdamOptimizer(LR_A).minimize(a_loss, var_list=self.ae_params)

        self.sess.run(tf.compat.v1.global_variables_initializer())

    def choose_action(self, s):
        return self.sess.run(self.a, {self.S: s[np.newaxis, :]})[0]

    def learn(self, replace):
        if self.a_replace_counter % REPLACE_ITER_A == 0 or replace:
            self.sess.run([tf.compat.v1.assign(t, e) for t, e in zip(self.at_params, self.ae_params)])

        if self.c_replace_counter % REPLACE_ITER_C == 0 or replace:
            self.sess.run([tf.compat.v1.assign(t, e) for t, e in zip(self.ct_params, self.ce_params)])

        self.a_replace_counter += 1;
        self.c_replace_counter += 1

        if self.pointer < MEMORY_CAPACITY:
            indices = np.random.choice(self.pointer, size=BATCH_SIZE)
        else:
            indices = np.random.choice(MEMORY_CAPACITY, size=BATCH_SIZE)
        bt = self.memory[indices, :]
        bs = bt[:, :self.s_dim]
        ba = bt[:, self.s_dim: self.s_dim + self.a_dim]
        br = bt[:, -self.s_dim - 1: -self.s_dim]
        bs_ = bt[:, -self.s_dim:]

        self.sess.run(self.atrain, {self.S: bs})
        self.sess.run(self.ctrain, {self.S: bs, self.a: ba, self.R: br, self.S_: bs_})  # {}place holder 处理输入

    def store_transition(self, s, a, r, s_):
        transition = np.hstack((s, a, [r], s_))
        index = self.pointer % MEMORY_CAPACITY  # replace the old memory with new memory
        self.memory[index, :] = transition
        self.pointer += 1

    def _build_a(self, s, scope, trainable):
        with tf.compat.v1.variable_scope(scope):
            net = tf.compat.v1.layers.dense(s, 30, activation=tf.nn.relu, name='l1', trainable=trainable)
            a = tf.compat.v1.layers.dense(net, self.a_dim, activation=tf.nn.tanh, name='a', trainable=trainable)
            return tf.compat.v1.multiply(a, self.a_bound, name='scaled_a')

    def _build_c(self, s, a, scope, trainable):
        with tf.compat.v1.variable_scope(scope):
            n_l1 = 30
            w1_s = tf.compat.v1.get_variable('w1_s', [self.s_dim, n_l1], trainable=trainable)
            w1_a = tf.compat.v1.get_variable('w1_a', [self.a_dim, n_l1], trainable=trainable)
            b1 = tf.compat.v1.get_variable('b1', [1, n_l1], trainable=trainable)
            net = tf.compat.v1.nn.relu(tf.matmul(s, w1_s) + tf.matmul(a, w1_a) + b1)
            return tf.compat.v1.layers.dense(net, 1, trainable=trainable)  # Q(s,a)

    def savenet(self, file):
        saver = tf.compat.v1.train.Saver()
        saver.save(self.sess, 'save/' + file + '.ckpt')

    def savenet_Test(self, file, index):
      saver = tf.compat.v1.train.Saver()
      saver.save(self.sess, 'save/'+'folder'+index+"/" + file + '.ckpt')

    def restore(self, file):
        saver = tf.train.Saver()
        saver.restore(self.sess, 'Models/' + file + '.ckpt')
        # saver.restore(self.sess, 'save/' + file + '.ckpt')

env = Env(TTC_threshold)
s_dim = env.n_features
a_dim = env.n_actions
a_bound = env.action_Bound

# load training data
train = sio.loadmat('/content/Velocity_control/trainSet.mat')['calibrationData']
test = sio.loadmat('/content/Velocity_control/testSet.mat')['validationData']
trainNum = train.shape[0]
testNum = test.shape[0]
print('Number of training samples:', trainNum)
print('Number of validate samples:', testNum)

# # Stop distance collision avoidance
n_run = 3
rolling_window = 100  # 100 car following events, average score
result = []

sim,speed,dis,acc,rel=Fuzzy_reward(train)
wandb.init()
for run in [base_name]:
    tf.compat.v1.reset_default_graph()
    # name is the name of the experiment, CA is whether use collision avoidance
    ddpg = DDPG(a_dim, s_dim, a_bound)

    # training part
    max_rolling_score = np.float('-inf')
    max_score = np.float('-inf')
    var = 3
    collision_train = 0
    headway_score = np.zeros(total_episode)
    episode_score = np.zeros(total_episode)  # average score of each car following event
    rolling_score = np.zeros(total_episode)
    cum_collision_num = np.zeros(total_episode)

    score_safe = np.zeros(total_episode)
    score_efficiency = np.zeros(total_episode)
    score_comfort = np.zeros(total_episode)

    for i in tqdm(range(total_episode)):
        car_fol_id = random.randint(0, trainNum - 1)
        data = train[car_fol_id, 0]
        s = env.reset(data)
        score = 0
        score_s, score_e, score_c, score_h = 0, 0, 0, 0  # part objective scores

        while True:
            # Add exploration noise
            a = ddpg.choose_action(s)[0]

            a = np.clip(np.random.normal(a, var), -a_bound, a_bound)  # add randomness to action selection for exploration
            if i<100 and a >-2:
              a-=1
            if i>10000:
                # add collision avoidance guidance
                state = env.s
                space, svSpd, relSpd = state
                lvSpd = svSpd + relSpd
                RT = 1  # reaction time
                SD = svSpd * RT + (svSpd ** 2 - lvSpd ** 2) / (2 * a_bound)

                if space < SD:
                    a = - a_bound

            s_, r, done, r_info,_ = env.step(a,sim,speed,dis,acc,rel)
            # sys.stdout.write(f'\r TTC: {r_info[0]}, hdw: {r_info[1]}, jerk: {r_info[2]}, reward: {r_info[3]}')
            # sys.stdout.flush()

            ddpg.store_transition(s, a, r, s_)
            replace = False
            if ddpg.pointer > MEMORY_CAPACITY:
                var *= .9995  # decay the action randomness
                ddpg.learn(replace)

            s = s_
            score += r
            score_s += r_info[0]
            score_e += r_info[1]
            score_c += r_info[2]
            # score_h += r_info[7]
            if done:
                duration = data.shape[0]
                score /= duration  # normalize with respect to car-following length
                score_s /= duration
                score_e /= duration
                score_c /= duration
                # score_h /= duration
                if env.isCollision == 1:
                    # print("TTC",score_s,"Headway",score_e,"Jerk",score_c)
                    collision_train += 1
                break

        # record episode results
        episode_score[i] = score
        score_safe[i] = score_s
        # headway_score[i] = score_h
        score_efficiency[i] = score_e
        score_comfort[i] = score_c
        rolling_score[i] = np.mean(episode_score[max(0, i - rolling_window + 1):i + 1])
        cum_collision_num[i] = collision_train

        if max_score < score:
            max_score = score

        if rolling_score[i] > max_rolling_score:
            max_rolling_score = rolling_score[i]
            # save network parameters
            ddpg.savenet(f'model_{run}')


        sys.stdout.write(
            f'''\r Run {run}, Episode {i}, Score: {score:.2f}, Rolling score: {rolling_score[i]:.2f}, Max score: {max_score:.2f}, Max rolling score: {max_rolling_score:.2f}, TTC: {score_safe[i]:.2f},Headway: {score_efficiency[i]:.2f}，Jerk: {score_comfort[i]:.2f} ,collisions: {collision_train}   ''')
        sys.stdout.flush()
        wandb.log({'Reward':score},step=i)
        wandb.log({'TTC_curve':score_safe[i]},step=i)
        wandb.log({'Headway_curve':score_efficiency[i]},step=i)
        wandb.log({'Jerk_curve':score_comfort[i]},step=i)
        wandb.log({'collisions':collision_train},step=i)
        wandb.log({'Rolling score':rolling_score[i]},step=i)
        if i%200==0 and i>=300:
            Model_test(ddpg,testNum,test,sim,speed,dis,acc,rel)
            print("Save_Model:",f'model_{run}_{i/100}')
            ddpg.savenet(f'{i/99}_model_{run}')
            # make_zip('/content/save','/content/Car-following MyModel_10.'+str((i+1)/1000)+'_dynamic_HDW.zip')
    # save results
    result.append([episode_score, rolling_score, cum_collision_num, score_safe, score_efficiency, score_comfort])

np.save(f'result_{run}.npy', result)

Number of training samples: 938
Number of validate samples: 403


wandb: Currently logged in as: ryanliu19990111. Use `wandb login --relogin` to force relogin


  0%|          | 0/3000 [00:00<?, ?it/s]

 Run origin_4.0002_NOCA, Episode 400, Score: 3.58, Rolling score: 3.79, Max score: 6.79, Max rolling score: 5.29, TTC: 1.10,Headway: 2.06，Jerk: 0.06 ,collisions: 6   

  0%|          | 0/403 [00:00<?, ?it/s]

...

 Test with test data, Episode 402, Score: 6.68, Rolling score: 6.79, Max score: 8.05, Max rolling score: 7.26, collisions: 0   Running time: 206.8816487789154
Running time per event: 0.5133539671933385
-------------------------------------------------- MeasureResult --------------------------------------------------
collisions: 0 Average TTC: 174.23295716519715 Average Headway: 1.4975793916060636 Average Jerk: 5.274108523686553 Average Distance: 16.397286208109385 Average Rel_Speed: 1.1130446813757084

-------------------------------------------------- END --------------------------------------------------

Save_Model: model_origin_4.0002_NOCA_18.0
 Run origin_4.0002_NOCA, Episode 2000, Score: 7.19, Rolling score: 6.46, Max score: 8.20, Max rolling score: 6.97, TTC: -129.48,Headway: 1.37，Jerk: 0.02 ,collisions: 6   

  0%|          | 0/403 [00:00<?, ?it/s]

 Test with test data, Episode 402, Score: 6.66, Rolling score: 6.65, Max score: 7.91, Max rolling score: 7.10, collisions: 0   Running time: 208.35027647018433
Running time per event: 0.5169982046406559
-------------------------------------------------- MeasureResult --------------------------------------------------
collisions: 0 Average TTC: 165.94014141459783 Average Headway: 1.4228111694770376 Average Jerk: 5.205794688300859 Average Distance: 20.134642400733007 Average Rel_Speed: 1.398219468297432

-------------------------------------------------- END --------------------------------------------------

Save_Model: model_origin_4.0002_NOCA_20.0
 Run origin_4.0002_NOCA, Episode 2200, Score: 5.41, Rolling score: 6.33, Max score: 8.21, Max rolling score: 6.97, TTC: -133.02,Headway: 396970.61，Jerk: -0.06 ,collisions: 6   

  0%|          | 0/403 [00:00<?, ?it/s]

 Test with test data, Episode 402, Score: 3.79, Rolling score: 6.05, Max score: 8.09, Max rolling score: 7.13, collisions: 0   Running time: 208.05437326431274
Running time per event: 0.5162639535094609
-------------------------------------------------- MeasureResult --------------------------------------------------
collisions: 0 Average TTC: 220.47408819668613 Average Headway: 1.5088220022438177 Average Jerk: 3.15953612380957 Average Distance: 28.208262259606343 Average Rel_Speed: 2.2607105733821538

-------------------------------------------------- END --------------------------------------------------

Save_Model: model_origin_4.0002_NOCA_22.0
 Run origin_4.0002_NOCA, Episode 2241, Score: 6.23, Rolling score: 6.12, Max score: 8.21, Max rolling score: 6.97, TTC: -62.18,Headway: 1.70，Jerk: 0.02 ,collisions: 6   